In [ ]:
import os
import zipfile
import random
import shutil
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing import image
from PIL import Image
from shutil import copyfile
import matplotlib.pyplot as plt

In [ ]:
local_zip = './waste-dataset.zip'
extractdir = "./"
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(extractdir)
zip_ref.close()

In [ ]:
# Function split_data
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    # Shuffle list
    shuffled_source = random.sample(os.listdir(SOURCE), len(os.listdir(SOURCE)))
    # Find total number of files in training dir
    training_number = int(len(shuffled_source) * SPLIT_SIZE)
    i = 0
    target = TRAINING

    for item in shuffled_source:
        item_source = os.path.join(SOURCE, item)
        if os.path.getsize(item_source) == 0:
            print(f'{item} is zero length, so ignoring.')
        else:
            copyfile(item_source, os.path.join(target, item))
            i += 1

    # Switch copy target to TESTING
        if i == training_number:
            target = TESTING

In [ ]:
source_path = './waste-dataset/imgWaste'

source_path_O = os.path.join(source_path, 'O')
source_path_R = os.path.join(source_path, 'R')


# os.listdir returns a list containing all files under the given path
print(f"There are {len(os.listdir(source_path_O))} images of O.")
print(f"There are {len(os.listdir(source_path_R))} images of R.")

There are 13966 images of O.
There are 11111 images of R.


In [ ]:
# Define root directory
root_dir = './waste-dataset/data'

# Empty directory to prevent FileExistsError is the function is run several times
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

# GRADED FUNCTION: create_train_test_dirs
def create_train_test_dirs(root_path):
  ### START CODE HERE

  # HINT:
  # Use os.makedirs to create your directories with intermediate subdirectories
  # Don't hardcode the paths. Use os.path.join to append the new directories to the root_path parameter
  os.makedirs(os.path.join(root_path, 'training'))
  os.makedirs(os.path.join(f'{root_path}/training', 'O'))
  os.makedirs(os.path.join(f'{root_path}/training', 'R'))
  os.makedirs(os.path.join(root_path, 'testing'))
  os.makedirs(os.path.join(f'{root_path}/testing', 'O'))
  os.makedirs(os.path.join(f'{root_path}/testing', 'R'))
  ### END CODE HERE


try:
  create_train_test_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [ ]:
# Test your create_train_test_dirs function
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

./waste-dataset/data\testing
./waste-dataset/data\training
./waste-dataset/data\testing\O
./waste-dataset/data\testing\R
./waste-dataset/data\training\O
./waste-dataset/data\training\R


In [ ]:
# GRADED FUNCTION: split_data
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):

  ### START CODE HERE
  # Shuffle list
  shuffled_source = random.sample(os.listdir(SOURCE), len(os.listdir(SOURCE)))

  # Find total number of files in training dir
  training_number = int(len(shuffled_source) * SPLIT_SIZE)

  i = 0
  target = TRAINING

  for item in shuffled_source:
    item_source = os.path.join(SOURCE, item)
    if os.path.getsize(item_source) == 0:
      print(f'{item} is zero length, so ignoring.')
    else:
      copyfile(item_source, os.path.join(target, item))
      i += 1

    # Switch copy target to TESTING
    if i == training_number:
      target = TESTING
  ### END CODE HERE

In [ ]:
# Test your split_data function

# Define paths
O_SOURCE_DIR = "./waste-dataset/imgWaste/O"
R_SOURCE_DIR = "./waste-dataset/imgWaste/R"

TRAINING_DIR = "./waste-dataset/data/training/"
TESTING_DIR = "./waste-dataset/data/testing/"

TRAINING_O_DIR = os.path.join(TRAINING_DIR, "O/")
TESTING_O_DIR = os.path.join(TESTING_DIR, "O/")

TRAINING_R_DIR = os.path.join(TRAINING_DIR, "R/")
TESTING_R_DIR = os.path.join(TESTING_DIR, "R/")

# Empty directories in case you run this cell multiple times
if len(os.listdir(TRAINING_O_DIR)) > 0:
  for file in os.scandir(TRAINING_O_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_R_DIR)) > 0:
  for file in os.scandir(TRAINING_R_DIR):
    os.remove(file.path)
if len(os.listdir(TESTING_O_DIR)) > 0:
  for file in os.scandir(TESTING_O_DIR):
    os.remove(file.path)
if len(os.listdir(TESTING_R_DIR)) > 0:
  for file in os.scandir(TESTING_R_DIR):
    os.remove(file.path)

# Define proportion of images used for training
split_size = .9

# Run the function
# NOTE: Messages about zero length images should be printed out
split_data(O_SOURCE_DIR, TRAINING_O_DIR, TESTING_O_DIR, split_size)
split_data(R_SOURCE_DIR, TRAINING_R_DIR, TESTING_R_DIR, split_size)

# Check that the number of images matches the expected output
print(f"\n\nThere are {len(os.listdir(TRAINING_O_DIR))} images of O for training")
print(f"There are {len(os.listdir(TRAINING_R_DIR))} images of R for training")
print(f"There are {len(os.listdir(TESTING_O_DIR))} images of O for testing")
print(f"There are {len(os.listdir(TESTING_R_DIR))} images of R for testing")



There are 12569 images of O for training
There are 9999 images of R for training
There are 1397 images of O for testing
There are 1112 images of R for testing


In [ ]:
# GRADED FUNCTION: train_val_generators
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  ### START CODE HERE

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  train_datagen = ImageDataGenerator(rescale = 1./255.)

  # Pass in the appropiate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=32,
                                                      class_mode='binary',
                                                      target_size=(64, 64))

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  validation_datagen = ImageDataGenerator(rescale = 1./255.)

  # Pass in the appropiate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=32,
                                                                class_mode='binary',
                                                                target_size=(64, 64))
  ### END CODE HERE
  return train_generator, validation_generator

In [ ]:
# Test your generators
train_generator, validation_generator = train_val_generators(TRAINING_DIR, TESTING_DIR)

Found 22568 images belonging to 2 classes.
Found 2509 images belonging to 2 classes.


In [ ]:
mobilenetv1 = "https://tfhub.dev/google/imagenet/mobilenet_v1_025_224/classification/5"
def create_model():
  # DEFINE A KERAS MODEL TO CLASSIFY CATS V DOGS
  # USE AT LEAST 3 CONVOLUTION LAYERS

  ### START CODE HERE
  tl_layer = hub.KerasLayer(mobilenetv1, input_shape=(64, 64, 3), trainable=False)
  model = tf.keras.models.Sequential([
      # # Note the input shape is the desired size of the image 150x150 with 3 bytes color
      # tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
      # tf.keras.layers.MaxPooling2D(2,2),
      # tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      # tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
      # #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      # #tf.keras.layers.MaxPooling2D(2,2),
      # # Flatten the results to feed into a DNN
      # tf.keras.layers.Flatten(),
      # # 512 neuron hidden layer
      tl_layer,
      tf.keras.layers.Dense(128, activation='relu'),
      # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])


  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

  ### END CODE HERE

  return model

In [ ]:
# Get the untrained model
model = create_model()

# Train the model
# Note that this may take some time.
history = model.fit(train_generator,
                    epochs=20,
                    steps_per_epoch = 705,
                    validation_steps = 78,
                    validation_data=validation_generator)

Epoch 1/20
705/705 [==============================] - 154s 215ms/step - loss: 0.5116 - accuracy: 0.8471 - val_loss: 0.3054 - val_accuracy: 0.8786
Epoch 2/20
705/705 [==============================] - 31s 44ms/step - loss: 0.3010 - accuracy: 0.8810 - val_loss: 0.2930 - val_accuracy: 0.8838
Epoch 3/20
705/705 [==============================] - 29s 41ms/step - loss: 0.2766 - accuracy: 0.8879 - val_loss: 0.2777 - val_accuracy: 0.8862
Epoch 4/20
705/705 [==============================] - 26s 37ms/step - loss: 0.2632 - accuracy: 0.8944 - val_loss: 0.2928 - val_accuracy: 0.8866
Epoch 5/20
705/705 [==============================] - 26s 37ms/step - loss: 0.2541 - accuracy: 0.8976 - val_loss: 0.2875 - val_accuracy: 0.8926
Epoch 6/20
705/705 [==============================] - 26s 37ms/step - loss: 0.2424 - accuracy: 0.9027 - val_loss: 0.2921 - val_accuracy: 0.8846
Epoch 7/20
705/705 [==============================] - 26s 37ms/step - loss: 0.2313 - accuracy: 0.9074 - val_loss: 0.3042 - val_accurac

In [ ]:
# organic image
test_image = image.load_img('./waste-dataset/data/testing/O/O_19.jpg',target_size = (64, 64))

In [ ]:
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

1/1 [==============================] - 0s 396ms/step


In [ ]:
train_generator.class_indices

{'O': 0, 'R': 1}

In [ ]:
if result[0][0] == 1:
    prediction = 'Recyclable'
else:
    prediction = 'Organic'

prediction


'Organic'

In [ ]:
# run_model = tf.function(lambda x: model(x))

# concrete_funct = run_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))
# tf.saved_model.save(
#     model, 'tf2-save', signatures={
#         tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY: concrete_funct})
!mkdir -p saved_model
model.save('./tf2-save')

INFO:tensorflow:Assets written to: ./tf2-save\assets


INFO:tensorflow:Assets written to: ./tf2-save\assets


In [ ]:
saved_model_dir = './tf2-save'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)